In [ ]:
import logging #Reports information as the code is run
import os
import sys
import shutil
import tempfile #makes temporary file directories
import pandas as pd

from glob import glob #function that's used to search for files that match a specific file pattern or name.

import re
import random

import torch
torch.manual_seed(42)
from PIL import Image
from torch.utils.tensorboard import SummaryWriter #Used to make visual bar graphs of epoch running

import numpy as np
import matplotlib.pyplot as plt
import io

from datetime import date
import warnings

from sklearn.model_selection import KFold

In [ ]:
pip install monai

In [ ]:
import monai
from monai.data import ArrayDataset, Dataset, decollate_batch, DataLoader, list_data_collate
#ArrayDataset -> Used to easily load images and labels as datasets

    # # Create ArrayDataset
    # dataset = ArrayDataset(images, labels, transform=image_transform, seg_transform=label_transform)
#decollate_batch extracts a single image and segmentation from a batch
#DataLoader loads the data in batches and organizes for the computer to operate

from monai.inferers import sliding_window_inference #enables inference on large medical images by dividing them into smaller overlapping windows
from monai.metrics import DiceMetric, MeanIoU #Calcualtes Dice loss 2*Union/(A+B)
from monai.transforms import (
    Activations, #End of neural network activaitons, that turns values from the CNN into probabilities
    AsDiscrete, #Turns probabilites into discrete values
    Compose, #Sums together image transformations
    LoadImage, #Loads image from file directory, converts to tensor format
    LoadImaged,
    RandRotate90,
    RandWeightedCropd,
    RandSpatialCrop,
    RandCropByLabelClasses,
    RandGaussianNoised,
    RandSpatialCropSamples,
    RandRotated,
    # AddChanneld,
    ScaleIntensityRanged,
    RandCropByPosNegLabeld,
    ScaleIntensityd,
    RandRotate90d,
    EnsureChannelFirst,
    EnsureChannelFirstd,
    ToTensord,
    RandCropByPosNegLabel,
    ScaleIntensity,
    AsDiscreted,
    RandRotated,
    RandSpatialCropSamplesd,
    RandSpatialCropd,
    Resized
)
from monai.visualize import plot_2d_or_3d_image
from torch.nn.functional import softmax

In [ ]:
pip install boxsdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [ ]:
import boxsdk
from boxsdk import Client, OAuth2

# Define client ID, client secret, and developer token.
ACCESS_TOKEN = "ACCESS_TOKEN"
CLIENT_ID = "CLIENT_ID"
CLIENT_SECRET = "CLIENT_SECRET"

# Create OAuth2 object. It's already authenticated, thanks to the developer token.
oauth2 = OAuth2(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN)

client = Client(oauth2)

In [ ]:
def navigTo(folder, file_name):
    items = folder.get_items()
    for item in items:
        if (item.name == file_name):
            return item
    return -1

def navig(client, file_names):
    folder_init = client.root_folder()
    for i in range(len(file_names)):
        folder_init = navigTo(folder_init, file_names[i])
    return folder_init

In [ ]:
# img_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'AX_T1_train_img.npy'])
# seg_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'AX_T1_train_seg.npy'])
# fore_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'AX_T1_train_fore.npy'])
# ID_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'AX_T1_train_id.npy'])

img_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'POST_AX_T1_FS_train_img.npy'])
seg_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'POST_AX_T1_FS_train_seg.npy'])
fore_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'POST_AX_T1_FS_train_fore.npy'])
ID_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'cross_validation', 'POST_AX_T1_FS_train_id.npy'])

# img_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'POST_AX_T1_FS_scans.npy'])
# seg_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'POST_AX_T1_FS_segmentations.npy'])
# fore_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'POST_AX_T1_FS_segmentations_foreground.npy'])
# ID_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'POST_AX_T1_FS_IDs.npy'])

# Obtain numpy arrays from box drive
image_byte_io = io.BytesIO(img_file.get().content())
img = np.load(image_byte_io)
del image_byte_io

# Obtain numpy arrays from box drive
seg_byte_io = io.BytesIO(seg_file.get().content())
seg = np.load(seg_byte_io)
del seg_byte_io

# Obtain numpy arrays from box drive
fore_byte_io = io.BytesIO(fore_file.get().content())
fore = np.load(fore_byte_io)
del fore_byte_io

# Obtain numpy arrays from box drive
ID_io = io.BytesIO(ID_file.get().content())
IDs = np.load(ID_io)
del ID_io

POST_AX_T1_FS_data = {"img": img, "seg": seg, "fore": fore, "id": IDs}

In [ ]:
def upload_directory_to_box(client, local_directory, box_folder):
    for root, dirs, files in os.walk(local_directory):
        relative_path = os.path.relpath(root, local_directory)
        current_box_folder = box_folder

        if relative_path != '.':  # Ignore the root directory itself
            folder_name = os.path.basename(root)
            items = current_box_folder.get_items()
            existing_folder = next((item for item in items if item.type == 'folder' and item.name == folder_name), None)

            if existing_folder:
                current_box_folder = client.folder(existing_folder.id)  # Access folder by ID
            else:
                current_box_folder = client.folder(current_box_folder.id).create_subfolder(folder_name)  # Create subfolder from current folder

        # Upload files to the current folder
        for file_name in files:
            file_path = os.path.join(root, file_name)
            existing_files = current_box_folder.get_items()
            existing_file = next((item for item in existing_files if item.name == file_name and item.type == 'file'), None)

            if existing_file:
                existing_file.delete()

            with open(file_path, 'rb') as file_stream:
                current_box_folder.upload_stream(file_stream, file_name)

In [ ]:
def load_model(client, file_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    box_model = navig(client, file_name)
    image_byte_io = io.BytesIO(box_model.get().content())

    if (device.__str__() == "cpu"):
        model_params = torch.load(image_byte_io, map_location="cpu")
    else:
        model_params = torch.load(image_byte_io)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=3,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=2,
    ).to(device)

    model.load_state_dict(model_params)
    model.eval()

    return model

def try_model(model, img, seg = None, plot = True, crf = False):

    img = img.transpose((1, 0))

    if (plot):
        plt.imshow(img)
        plt.figure()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = torch.tensor(img)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.to(device)
    inputs = inputs.float()
    inputs = ScaleIntensity()(inputs)

    val_outputs = sliding_window_inference(inputs, (128, 128), 1, model) #gets the output by performing sliding_window inference on the image
    if (crf):
        val_outputs = apply_crf(val_outputs, image = inputs)
    val_outputs_numpy = val_outputs.cpu().detach().numpy()
    if (plot):
        plt.imshow(val_outputs_numpy[0][0])
        plt.figure()
        plt.imshow(val_outputs_numpy[0][1])
        plt.figure()
        plt.imshow(val_outputs_numpy[0][2])
        plt.figure()

    post_trans = Compose([Activations(softmax=True), AsDiscrete(argmax=True, to_onehot=3)])

    transform_output = post_trans(val_outputs[0])
    transform_output = transform_output.cpu().detach().numpy()
    transform_output_transposed = transform_output.transpose((2, 1, 0))
    if (plot):
        plt.imshow(transform_output_transposed)

    if (not seg is None):
        seg = seg.transpose((1, 0))
        if (plot):
            plt.figure()
            plt.imshow(seg)
        return {"transformed_output": transform_output_transposed, "output": val_outputs_numpy[0], "GT": seg}
    else:
        return {"transformed_output": transform_output_transposed, "output": val_outputs_numpy[0]}

In [ ]:
from typing import Dict

class CustomRandCropByPosNegLabeld(RandCropByPosNegLabeld):
    def __init__(self, keys, label_key, spatial_size, pos=1, neg=1, num_samples_pos=6, num_samples_neg=1, image_threshold=0):
        super().__init__(keys=keys, label_key=label_key, spatial_size=spatial_size, pos=pos, neg=neg, num_samples=(num_samples_pos+num_samples_neg)//2, image_threshold=image_threshold)
        self.num_samples_pos = num_samples_pos
        self.num_samples_neg = num_samples_neg
        self.spatial_size = spatial_size
        self.pos = pos
        self.neg = neg
        self.image_threshold = image_threshold

    def __call__(self, data: Dict, lazy: bool = False):
        # Check if there are any foreground labels in the segmentation mask
        label_data = data[self.label_key]
        contains_foreground = torch.any(label_data[0] == 0)

        # Adjust num_samples based on foreground presence
        self.num_samples = self.num_samples_pos if contains_foreground else self.num_samples_neg
        pos_val = self.pos if contains_foreground else 0

        # Reinitialize the parent to accept updated num_samples
        super().__init__(self.keys, self.label_key, self.spatial_size, pos_val, self.neg, self.num_samples, self.image_threshold)

        return super().__call__(data)

In [ ]:
ROI_SIZE = 128
NUM_CLASSES = 3

train_transforms_pos_neg = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        EnsureChannelFirstd(keys=["img", "seg"]),
        ScaleIntensityd(keys=["img"]),
        AsDiscreted(keys="seg", to_onehot=NUM_CLASSES),
        RandRotated(keys=["img", "seg"], range_x = 15*np.pi/180, prob=0.5), #90 degree rotation
        RandGaussianNoised(keys=["img"], prob = 0.25, mean = 0.0, std = 0.05),
        CustomRandCropByPosNegLabeld(
            keys=["img", "seg"], label_key="seg", spatial_size=[ROI_SIZE, ROI_SIZE], pos=2, neg=1, num_samples_pos=8, num_samples_neg=1
        ), #Crop, with a 1:1 ratio of patches with and without a lacrimal gland
    ]
)

train_transforms_spatial = Compose(
    [
        LoadImaged(keys=["img", "seg", "map"]),
        EnsureChannelFirstd(keys=["img", "seg", "map"]),
        ScaleIntensityd(keys=["img"]),
        AsDiscreted(keys="seg", to_onehot=NUM_CLASSES),
        RandRotated(keys=["img", "seg", "map"], range_x = 15*np.pi/180, prob=0.5), #90 degree rotation
        RandGaussianNoised(keys=["img"], prob = 0.25, mean = 0.0, std = 0.05),
        RandWeightedCropd(
            keys=["img", "seg"], w_key="map", spatial_size=[ROI_SIZE, ROI_SIZE], num_samples = 4
        ), #Crop, with a 1:1 ratio of patches in the X-Y range of the lacrimal gland
    ]
)

train_transforms_spatial_resize = Compose(
    [
        LoadImaged(keys=["img", "seg", "map"]),
        EnsureChannelFirstd(keys=["img", "seg", "map"]),
        ScaleIntensityd(keys=["img"]),
        AsDiscreted(keys="seg", to_onehot=NUM_CLASSES),
        RandRotated(keys=["img", "seg", "map"], range_x = 15*np.pi/180, prob=0.5), #90 degree rotation
        RandGaussianNoised(keys=["img"], prob = 0.25, mean = 0.0, std = 0.05),
        RandWeightedCropd(
            keys=["img", "seg"], w_key="map", spatial_size=[int(ROI_SIZE*1.25), int(ROI_SIZE*1.25)], num_samples = 4
        ),
        RandSpatialCropd(
            keys=["img", "seg"], roi_size = [int(ROI_SIZE*0.8), int(ROI_SIZE*0.8)], max_roi_size = [int(ROI_SIZE*1.25), int(ROI_SIZE*1.25)], random_center = False, random_size = True
        ),
        Resized(
            keys=["img", "seg"], spatial_size=[ROI_SIZE, ROI_SIZE]
        ), #Crop, with a 1:1 ratio of patches with and without a lacrimal gland
    ]
)

post_trans = Compose(
    [
        Activations(softmax=True),
        AsDiscrete(argmax=True, to_onehot=3)
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["img", "seg"]),
        EnsureChannelFirstd(keys=["img", "seg"]),
        ScaleIntensityd(keys=["img"]),
        AsDiscreted(keys="seg", to_onehot=NUM_CLASSES),
    ]
)

In [ ]:
def create_experiment_folder(exp_name,
                             num_files,
                             max_epochs,
                             learning_rate,
                             use_pretrained_model,
                             loss_function,
                             split,
                             val_interval,
                             transform_type,
                             weight_map):
    directory_path = exp_name
    os.makedirs(directory_path, exist_ok=True)

    config_path = os.path.join(directory_path, "config")
    images_path = os.path.join(directory_path, "images")
    models_path = os.path.join(directory_path, "models")
    learning_data_path = os.path.join(directory_path, "learning_data")

    os.makedirs(config_path, exist_ok=True)
    os.makedirs(images_path, exist_ok=True)
    os.makedirs(models_path, exist_ok=True)
    os.makedirs(learning_data_path, exist_ok=True)

    for item in os.listdir(config_path):
        item_path = os.path.join(config_path, item)
        os.remove(item_path)

    config_file_name = os.path.join(config_path, "config.txt")
    with open(config_file_name, "w") as file:
        file.write("exp_name: "+str(exp_name)+"\n")
        file.write("num_files: "+str(num_files)+"\n")
        file.write("max_epochs: "+str(max_epochs)+"\n")
        file.write("learning_rate: "+str(learning_rate)+"\n")
        file.write("use_pretrained_model: "+str(use_pretrained_model)+"\n")
        file.write("loss_function: "+str(loss_function)+"\n")
        file.write("split: "+str(split)+"\n")
        file.write("val_interval: "+str(val_interval)+"\n")
        file.write("transform_type: "+str(transform_type)+"\n")
        file.write("weight_map: "+str(weight_map)+"\n")

    with open(config_file_name, "r") as file:
        print(file.read())

    for item in os.listdir(images_path):
        item_path = os.path.join(images_path, item)
        os.remove(item_path)

    for item in os.listdir(models_path):
        item_path = os.path.join(models_path, item)
        os.remove(item_path)

    for item in os.listdir(learning_data_path):
        item_path = os.path.join(learning_data_path, item)
        os.remove(item_path)

    return {"parent_path": exp_name, "images_path": images_path, "models_path": models_path, "learning_data_path": learning_data_path}

def load_model_params(direc):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if (direc == None):
        return None
    box_model = navig(client, direc)
    image_byte_io = io.BytesIO(box_model.get().content())
    if (device.__str__() == "cpu"):
        model_params = torch.load(image_byte_io, map_location="cpu")
    else:
        model_params = torch.load(image_byte_io)

    return model_params

def get_model(model_params):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    unet_model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=3,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
        num_res_units=3,
    ).to(device)
    model = unet_model
    # model = UNetWithCRF(unet_model, crf_iterations=5)
    if (model_params != None):
        model.load_state_dict(model_params)

    return model

def sort_image_files(file_list):
    # Sort the file list by extracting the number in "img##.png"
    sorted_files = sorted(file_list, key=lambda x: int(re.search(r'\d+', x[x.rfind("/")+1:]).group()))
    return sorted_files

def split_shuffle_data(images, segmentations, split):
    # Combine the lists and shuffle
    combined = list(zip(images, segmentations))

    random.seed(42)
    indices = list(range(len(combined)))
    random.shuffle(indices)
    combined = [combined[i] for i in indices]

    training_indices = indices[:int(len(indices)*split)]
    validation_indices = indices[int(len(indices)*split):]
    print("validation_indices:", validation_indices)

    # Use the shuffled indices to reorder the original list
    training = combined[:int(len(combined)*split)]
    validation = combined[int(len(combined)*split):]

    # Unzip the lists back to their original form
    training_images, training_segmentations = zip(*training)
    validation_images, validation_segmentations = zip(*validation)

    # Convert them back to lists (zip returns tuples)
    training_images = list(training_images)
    training_segmentations = list(training_segmentations)
    validation_images = list(validation_images)
    validation_segmentations = list(validation_segmentations)

    return training_images, training_segmentations, validation_images, validation_segmentations

def prepare_fore_map(fore, tempdir, weight_map):
    fore = fore.astype(np.float32)
    fore_right = fore[:, :, :fore.shape[2]//2]
    fore_left = fore[:, :, fore.shape[2]//2:]
    fore_right_sum = np.sum(fore_right, axis = (1, 2))[:, None, None]
    fore_right_sum[fore_right_sum == 0] = 1
    fore_left_sum = np.sum(fore_left, axis = (1, 2))[:, None, None]
    fore_left_sum[fore_left_sum == 0] = 1
    back_sum = np.sum(1-fore, axis = (1, 2))[:, None, None]

    fore = (1-fore) / back_sum * weight_map[0]
    fore_right = fore_right / fore_right_sum * weight_map[1]
    fore_left = fore_left / fore_left_sum * weight_map[2]
    fore[:, :, :fore.shape[2]//2] = fore[:, :, :fore.shape[2]//2] + fore_right
    fore[:, :, fore.shape[2]//2:] = fore[:, :, fore.shape[2]//2:] + fore_left
    fore = fore*((2**16-1)/np.max(fore))

    for i in range(len(fore)): #Make 40 such images
        Image.fromarray((fore[i]).astype("uint16")).save(os.path.join(tempdir, f"fore{i:d}.png"))
    foregrounds = sort_image_files(glob(os.path.join(tempdir, "fore*.png")))

    return foregrounds

def get_train_loader(training_images, training_segmentations, fore = None, transform_type = "spatial", tempdir = None, weight_map = None, num_workers = 2):
    possible_transforms = {"pos_neg": train_transforms_pos_neg, "spatial": train_transforms_spatial, "spatial_resize": train_transforms_spatial_resize}

    train_transforms = possible_transforms[transform_type]

    if (transform_type == "pos_neg"):
        train_files = [{"img": imgs, "seg": segs} for imgs, segs in zip(training_images, training_segmentations)]
    else:
        fore = fore.astype(np.float32)
        fore_right = fore[:, :, :fore.shape[2]//2]
        fore_left = fore[:, :, fore.shape[2]//2:]
        fore_right_sum = np.sum(fore_right, axis = (1, 2))[:, None, None]
        fore_right_sum[fore_right_sum == 0] = 1
        fore_left_sum = np.sum(fore_left, axis = (1, 2))[:, None, None]
        fore_left_sum[fore_left_sum == 0] = 1
        back_sum = np.sum(1-fore, axis = (1, 2))[:, None, None]

        fore = (1-fore) / back_sum * weight_map[0]
        fore_right = fore_right / fore_right_sum * weight_map[1]
        fore_left = fore_left / fore_left_sum * weight_map[2]
        fore[:, :, :fore.shape[2]//2] = fore[:, :, :fore.shape[2]//2] + fore_right
        fore[:, :, fore.shape[2]//2:] = fore[:, :, fore.shape[2]//2:] + fore_left
        fore = fore*((2**16-1)/np.max(fore))

        for i in range(len(training_images)): #Make 40 such images
            Image.fromarray((fore[i]).astype("uint16")).save(os.path.join(tempdir, f"fore{i:d}.png"))
        foregrounds = sort_image_files(glob(os.path.join(tempdir, "fore*.png")))

        train_files = [{"img": imgs, "seg": segs, "map": maps} for imgs, segs, maps in zip(training_images, training_segmentations, foregrounds)]

    train_ds = ArrayDataset(train_files, train_transforms) #Make a training data set using the transformations previously mentioned.
    train_loader = DataLoader(train_ds,
                              batch_size=20,
                              shuffle = True,
                              num_workers = num_workers,
                              collate_fn=list_data_collate,
                              pin_memory=torch.cuda.is_available(),)

    return train_loader

def get_val_loader(validation_images, validation_segmentations, num_workers = 8):
    val_files = [{"img": imgs, "seg": segs} for imgs, segs in zip(validation_images, validation_segmentations)]
    val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=8, num_workers=num_workers, collate_fn=list_data_collate)

    return val_loader

def print_train_data_attributes(train_loader):
    with torch.no_grad():
        background_images = 0
        foreground_images = 0
        for i, batch in enumerate(train_loader):
            segmentation_batch = batch["seg"][:, 0, :, :].cpu().detach().numpy()
            segmentation_batch = 1 - segmentation_batch
            segmentation_batch_sum = np.sum(segmentation_batch, axis = (1, 2))
            segmentation_batch_sum[segmentation_batch_sum >= 1] = 1
            segmentation_batch_sum[segmentation_batch_sum < 1] = 0
            foreground_images += np.sum(segmentation_batch_sum)
            background_images += len(segmentation_batch_sum) - np.sum(segmentation_batch_sum)
        print("Background_images:", int(background_images), "|", background_images/(foreground_images + background_images)*100, "%")
        print("Foreground_images:", int(foreground_images), "|", foreground_images/(foreground_images + background_images)*100, "%")

def pad_df(data):
    data_copy = data.copy()
    max_len = max(len(v) for v in data_copy.values())
    for k, v in data_copy.items():
        if len(v) < max_len:
            data_copy[k] = v + [np.nan] * (max_len - len(v))

    df = pd.DataFrame(data_copy)

    return df

def save_learning_data(folder, train_data, val_data):
    train_df = pad_df(train_data)
    val_df = pad_df(val_data)

    train_data_path = os.path.join(folder, "train_data.xlsx")
    val_data_path = os.path.join(folder, "val_data.xlsx")
    train_df.to_excel(train_data_path, index=False)
    val_df.to_excel(val_data_path, index=False)

def plot_learning_data(train_data, val_data, fold=0):
    loss_fig = plt.figure()
    plt.plot(train_data["epoch_"+str(fold)], train_data["loss_"+str(fold)], label="Training Loss")
    plt.plot(val_data["epoch_"+str(fold)], val_data["loss_"+str(fold)], label="Validation Loss")
    plt.xlabel("Epoch, Fold " + str(fold))
    plt.ylabel("Loss")
    plt.legend()

    metrics_fig = plt.figure()
    plt.plot(val_data["epoch_"+str(fold)], val_data["dice_"+str(fold)], label="Validation Dice")
    plt.plot(val_data["epoch_"+str(fold)], val_data["iou_"+str(fold)], label="Validation IOU")
    plt.xlabel("Epoch, Fold " + str(fold))
    plt.ylabel("Metric Value")
    plt.legend()

    return loss_fig, metrics_fig

def get_random_images(model, seg, img, num_samples = 12, max_rows = None, seed = None, crf = True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if (seed != None):
        np.random.seed(seed)

    upper_bound = len(img)

    unique_random_integers = np.random.choice(np.arange(upper_bound), size=num_samples, replace=False)

    num_columns = 1
    num_rows = num_samples
    if (max_rows != None):
        num_columns = num_samples // max_rows
        num_columns = num_columns + 1 if num_samples % max_rows != 0 else num_columns
        num_rows = max_rows

    fig, axes = plt.subplots(num_rows, 3*num_columns, figsize=(15*num_columns, 5 * num_rows))

    for i in range(-(-num_samples // num_rows)*num_rows):
        row = i % num_rows
        col = i // num_rows * 3

        if (i in range(num_samples)):
            index = unique_random_integers[i]
            output = try_model(model, img[index], plot = False, crf = crf)
            transformed_output = output["transformed_output"]
            axes[row, col+0].imshow(transformed_output)
            axes[row, col+1].imshow(seg[index])
            axes[row, col+2].imshow(img[index])

        axes[row, col+0].set_xticks([])
        axes[row, col+0].set_yticks([])
        axes[row, col+1].set_xticks([])
        axes[row, col+1].set_yticks([])
        axes[row, col+2].set_xticks([])
        axes[row, col+2].set_yticks([])

    # Add titles for each column
    column_titles = ['Model', 'Ground Truth', 'Scan'] * num_columns
    for ax, title in zip(axes[0], column_titles):  # Loop over the top row of axes
        ax.set_title(title, fontsize=34)

    plt.tight_layout()
    return fig

def copy_folder_to_parent(source_folder, parent_folder, ignore_string=None):
    # Ensure the source folder exists
    if not os.path.exists(source_folder):
        print(f"Source folder '{source_folder}' does not exist.")
        return

    folder_name = os.path.basename(source_folder.rstrip("/\\"))
    destination_folder = os.path.join(parent_folder, folder_name)

    os.makedirs(parent_folder, exist_ok=True)

    def ignore_func(dir, contents):
        ignored = []
        for item in contents:
            full_path = os.path.join(dir, item)
            if ignore_string and ignore_string in full_path:
                ignored.append(item)
        return ignored

    shutil.copytree(source_folder, destination_folder, ignore=ignore_func)
    print(f"Copied '{source_folder}' to '{destination_folder}', ignoring paths containing '{ignore_string}'.")

def save_experiment_to_box(exp_name, client, token = None):
    if (token != None):
        ACCESS_TOKEN = token
        CLIENT_ID = "010e91gvkgovmnwpgtp7etpti2ha4rs9"
        CLIENT_SECRET = "d2uSd1VmN631T9VT7xPJVguZmA0Cggq5"

        # Create OAuth2 object. It's already authenticated, thanks to the developer token.
        oauth2 = OAuth2(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN)

        client = Client(oauth2)

    current_date = date.today()
    formatted_date = current_date.strftime("%m_%d_%y")

    box_directory = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'Models', formatted_date])
    if (box_directory == -1):
        box_directory = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'Models'])
        box_directory = box_directory.create_subfolder(formatted_date)
    try:
        box_directory = box_directory.create_subfolder(exp_name)
    except:
        box_directory = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'Models', formatted_date, exp_name])
    box_folder = box_directory.get()
    upload_directory_to_box(client, exp_name, box_folder)

In [ ]:
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_softmax, create_pairwise_gaussian, create_pairwise_bilateral
import torch.nn.functional as F

ModuleNotFoundError: No module named 'pydensecrf'

In [ ]:
def apply_crf(x, image = None, iter = 5):
    if not (type(x) == torch.tensor):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        x = torch.tensor(x).to(device)
    h, w = x.shape[2:]
    num_classes = x.shape[1]

    refined_slices = []
    for i, x_slice in enumerate(x):
        # Convert x to softmax probabilities
        probs = F.softmax(x_slice, dim=0)
        probs = probs.cpu().detach().numpy()

        # Prepare unary potentials
        unary = unary_from_softmax(probs)
        unary = unary.reshape((num_classes, -1))  # (C, H*W)

        # Initialize the CRF
        d = dcrf.DenseCRF2D(w, h, num_classes)
        d.setUnaryEnergy(unary)

        # Add pairwise terms (Gaussian)
        d.addPairwiseGaussian(sxy=(3, 3), compat=3)  # Spatial smoothness
        if (image is not None):
            im = image[i].cpu().detach().numpy()
            im = np.ascontiguousarray(im)
            im = im * 255
            im = im.astype(np.uint8)
            im = np.moveaxis(im, 0, -1)
            im = np.tile(im, (1, 1, 3))
            d.addPairwiseBilateral(sxy=(80, 80), srgb=13, rgbim=im, compat=10)  # Appearance-based, add pairwise gaussian noise

        # Perform inference
        refined = np.array(d.inference(iter))  # Convert MatrixXf to NumPy array
        refined = refined.reshape((num_classes, h, w))
        refined = refined.argmax(axis=0)
        refined = np.eye(3)[refined]

        refined = refined.transpose(2, 0, 1)
        refined_slices.append(refined)

    refined_slices = torch.tensor(refined_slices).to(device)

    # Return the refined segmentation map (choose max probability class)
    return refined_slices

In [ ]:
def main(tempdir,
         data,
         exp_name,
         num_files,
         max_epochs,
         learning_rate = 0.0001,
         use_pretrained_model = None,
         loss_function = None,
         split = 0.8,
         val_interval = 2,
         transform_type = "pos_neg",
         weight_map = None,
         n_splits = 3,
         max_folds = None,
         drive_path = r"/content/drive/MyDrive/RPI/Research/Ginat Lab/Lacrimal Gland Project/Training Loops"):

    img = data["img"]
    seg = data["seg"]
    fore = data["fore"]

    print("Loaded Data")

    save_paths = create_experiment_folder(exp_name,
                                        num_files,
                                        max_epochs,
                                        learning_rate,
                                        use_pretrained_model,
                                        loss_function,
                                        split,
                                        val_interval,
                                        transform_type,
                                        weight_map)

    print("Created drive folders")

    torch.manual_seed(42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load data to tempdir
    print(f"generating synthetic data to {tempdir} (this may take a while)")
    if (num_files == None):
        num_files = len(img)
    for i in range(num_files): #Make 40 such images
        Image.fromarray((img[i]).astype("uint16")).save(os.path.join(tempdir, f"img{i:d}.png")) #Saves the image numpys in  temp directory as a png
        Image.fromarray((seg[i]).astype("uint8")).save(os.path.join(tempdir, f"seg{i:d}.png")) #Saves the segmentation numpys in  temp directory as a png
    images = sort_image_files(glob(os.path.join(tempdir, "img*.png"))) #Obtains the file directories of the images (sorted alphabetically), and saves them as a list
    segmentations = sort_image_files(glob(os.path.join(tempdir, "seg*.png")))

    if (transform_type == "pos_neg"):
        files = [{"img": imgs, "seg": segs} for imgs, segs in zip(images, segmentations)]
    else:
        print("Generating foregrounds")
        foregrounds = prepare_fore_map(fore, tempdir, weight_map)
        files = [{"img": imgs, "seg": segs, "map": maps} for imgs, segs, maps in zip(images, segmentations, foregrounds)]

    n_splits = n_splits
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    possible_transforms = {"pos_neg": train_transforms_pos_neg, "spatial": train_transforms_spatial, "spatial_resize": train_transforms_spatial_resize}
    train_transforms = possible_transforms[transform_type]

    train_data = {}
    val_data = {}
    for i in range(n_splits):
        train_data["epoch_"+str(i)] = []
        train_data["loss_"+str(i)] = []
        val_data["epoch_"+str(i)] = []
        val_data["loss_"+str(i)] = []
        val_data["dice_"+str(i)] = []
        val_data["iou_"+str(i)] = []

    print("train_data", train_data)
    print("val_data", val_data)
    dice_loss_function = monai.losses.DiceLoss(softmax=False, include_background=True)
    iou_loss_function = monai.losses.DiceLoss(softmax=False, include_background=True, jaccard = True)

    max_folds = n_splits if max_folds == None else max_folds

    for fold, (train_idx, val_idx) in enumerate(kfold.split(files)):

        if (fold >= max_folds):
            break

        print(f"\nFold {fold + 1}")

        # Load pretrained model from box drive, if applicable
        model_params = load_model_params(use_pretrained_model)

        # Create model and optimizer
        model = get_model(model_params)
        optimizer = torch.optim.Adam(model.parameters(), learning_rate)

        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer=optimizer,
            T_max=max_epochs,  # Number of epochs until restart
            eta_min=learning_rate * 1  # Minimum learning rate
        )

        print("Obtained model, set optimizer")

        train_files = [files[i] for i in train_idx]
        val_files = [files[i] for i in val_idx]

        train_ds = Dataset(train_files, train_transforms)
        train_loader = DataLoader(train_ds,
                                batch_size=20,
                                shuffle = True,
                                num_workers = 1,
                                collate_fn=list_data_collate,
                                pin_memory=torch.cuda.is_available(),)
        val_ds = Dataset(data=val_files, transform=val_transforms)
        val_loader = DataLoader(val_ds, batch_size=8, num_workers=1, collate_fn=list_data_collate)

        print("Got Data loaders")

        # Get train data attributes
        print_train_data_attributes(train_loader)

        # Start a PyTorch training loop
        best_metric = 0
        best_metric_epoch = -1

        print("Made loss functions")

        train_loader_length = len(train_loader)
        val_loader_length = len(val_loader)

        print("train_loader_length", train_loader_length)
        print("val_loader_length", val_loader_length)

        for epoch in range(max_epochs): #repeat through epochs
            print("\n"+"-" * 10)
            # print(f"epoch {epoch + 1}/{max_epochs}: Learning Rate = ") #Print epoch number
            print(f"epoch {epoch + 1}/{max_epochs}: Learning Rate = {scheduler.get_last_lr()[0]:.7f}") #Print epoch number

            model.train()
            print("Set model to train mode.")
            train_loss = 0
            step = 0

            print("Loading batch from train loader.")
            for batch_data in train_loader: #load each batch from the data loader
                print("batch loaded")
                if (step == 0):
                    print("training: ", end =  "")
                perc_value = step / train_loader_length * 100
                print(f"{perc_value:.2f}", end = "% - ")

                print("Loading labels to device")
                inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
                optimizer.zero_grad()
                print("Calculating model output")
                outputs = model(inputs)

                loss = loss_function(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                step += 1

            scheduler.step()
            print("100%")

            train_data["epoch_"+str(fold)].append(epoch + 1)
            train_data["loss_"+str(fold)].append(train_loss / step)
            print("Training Loss:", train_loss / step)

            if (epoch + 1) % val_interval == 0: #Every 2nd epoch
                model.eval()
                with torch.no_grad():

                    val_loss = 0
                    dice_metric = 0
                    iou_metric = 0

                    step = 0
                    curr_percent = 10

                    print("\nvalidation: ", end =  "")
                    for val_batch in val_loader:
                        perc_value = step / val_loader_length * 100
                        print(f"{perc_value:.2f}", end = "% - ")

                        val_images, val_labels = val_batch["img"].to(device), val_batch["seg"].to(device)

                        roi_size = (128, 128)
                        sw_batch_size = 4
                        val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)

                        loss = loss_function(val_outputs, val_labels)
                        val_loss += loss.item()
                        step += 1

                        val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                        val_outputs = torch.stack(val_outputs)

                        dice_metric += 1 - dice_loss_function(val_outputs, val_labels).item()
                        iou_metric += 1 - iou_loss_function(val_outputs, val_labels).item()
                    print("100%")

                    val_data["epoch_"+str(fold)].append(epoch + 1)
                    val_data["loss_"+str(fold)].append(val_loss / step)
                    print("Validation Loss:", val_loss / step)
                    val_data["dice_"+str(fold)].append(dice_metric / step)
                    val_data["iou_"+str(fold)].append(iou_metric / step)
                    print("Validation DICE:", dice_metric / step)
                    print("Validation IOU:", iou_metric / step)

                    if (dice_metric / step > best_metric):
                        best_metric = dice_metric / step
                        best_metric_epoch = epoch
                        torch.save(model.state_dict(), os.path.join(save_paths["models_path"], "best_metric_model_fold_"+str(fold)+".pth"))
                    torch.save(model.state_dict(), os.path.join(save_paths["models_path"], "most_recent_model_fold_"+str(fold)+".pth"))

                    fig = get_random_images(model, seg, img, num_samples = 12, max_rows = 6, seed = 42, crf = False)
                    fig.savefig(os.path.join(save_paths["images_path"], f"sample_images_epoch_{epoch:03d}.jpg"))
                    plt.close(fig)

                    save_learning_data(save_paths["learning_data_path"], train_data, val_data)
                    loss_fig, metrics_fig = plot_learning_data(train_data, val_data, fold=fold)
                    loss_fig.savefig(os.path.join(save_paths["learning_data_path"], f"loss_data.jpg"))
                    metrics_fig.savefig(os.path.join(save_paths["learning_data_path"], f"metric_data.jpg"))
                    plt.close(loss_fig)
                    plt.close(metrics_fig)

        print(f"train completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

    if (drive_path != None):
        copy_folder_to_parent(save_paths["parent_path"], drive_path, "images")

In [ ]:
warnings.filterwarnings("ignore", message="Num foregrounds 0, Num backgrounds .* setting `pos_ratio` to 0.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with tempfile.TemporaryDirectory() as tempdir:
    main(tempdir,
         POST_AX_T1_FS_data,
         exp_name = "POST_AX_T1_FS_cross_validation_scheduledLR_refined_spatial_targetted_sampling_resize_DICE_wo_crf_3_res_units",
         num_files = None,
         max_epochs = 300,
         learning_rate = 0.0001,
         use_pretrained_model = None,
         loss_function = monai.losses.DiceLoss(softmax=True, include_background=True, reduction = "mean"),
         split = 0.9,
         val_interval = 10,
         transform_type = "spatial_resize",
         weight_map = [2, 1, 1],
         n_splits = 5,
         max_folds = 1,
         drive_path = r"/content/drive/MyDrive/RPI/Research/Ginat Lab/Lacrimal Gland Project/Training Loops")

Streaming output truncated to the last 5000 lines.
65.22% - Loading labels to device
Calculating model output
batch loaded
67.39% - Loading labels to device
Calculating model output
batch loaded
69.57% - Loading labels to device
Calculating model output
batch loaded
71.74% - Loading labels to device
Calculating model output
batch loaded
73.91% - Loading labels to device
Calculating model output
batch loaded
76.09% - Loading labels to device
Calculating model output
batch loaded
78.26% - Loading labels to device
Calculating model output
batch loaded
80.43% - Loading labels to device
Calculating model output
batch loaded
82.61% - Loading labels to device
Calculating model output
batch loaded
84.78% - Loading labels to device
Calculating model output
batch loaded
86.96% - Loading labels to device
Calculating model output
batch loaded
89.13% - Loading labels to device
Calculating model output
batch loaded
91.30% - Loading labels to device
Calculating model output
batch loaded
93.48% - Load

FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/RPI/Research/Ginat Lab/Lacrimal Gland Project/Training Loops/POST_AX_T1_FS_cross_validation_scheduledLR_refined_spatial_targetted_sampling_resize_DICE_wo_crf_3_res_units'

In [ ]:
def save_experiment_to_box(exp_name, client, token = None):
    if (token != None):
        ACCESS_TOKEN = token
        CLIENT_ID = "010e91gvkgovmnwpgtp7etpti2ha4rs9"
        CLIENT_SECRET = "d2uSd1VmN631T9VT7xPJVguZmA0Cggq5"

        # Create OAuth2 object. It's already authenticated, thanks to the developer token.
        oauth2 = OAuth2(CLIENT_ID, CLIENT_SECRET, access_token=ACCESS_TOKEN)

        client = Client(oauth2)

    current_date = date.today()
    formatted_date = current_date.strftime("%m_%d_%y")

    if ("/" in exp_name):
        exp_folder_name = exp_name.split("/")[-1]
    else:
        exp_folder_name = exp_name

    box_directory = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'Models', formatted_date])
    if (box_directory == -1):
        box_directory = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'Models'])
        box_directory = box_directory.create_subfolder(formatted_date)
    try:
        box_directory = box_directory.create_subfolder(exp_folder_name)
    except:
        box_directory = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'Models', formatted_date, exp_name])
    box_folder = box_directory.get()
    upload_directory_to_box(client, exp_name, box_folder)

In [ ]:
file = r"/content/drive/MyDrive/RPI/Research/Ginat Lab/Lacrimal Gland Project/Training Loops/POST_AX_T1_FS_cross_validation_scheduledLR_refined_spatial_targetted_sampling_resize_DICE_wo_crf"
file = r"POST_AX_T1_FS_cross_validation_scheduledLR_refined_spatial_targetted_sampling_resize_DICE_wo_crf_3_res_units"
save_experiment_to_box(file, client, token = "qj2LNaZPX6frtzqfFT4bavw95zIwMd89")

In [ ]:
def get_ordered_images(model, seg, img, scan_IDs, sample_index = 0, max_rows = None, seed = None, crf = True, plot = True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dice_loss_function = monai.losses.DiceLoss(softmax=False, include_background=True)

    dice_metrics = []
    true_false_positives = []

    unique_scan_IDs = np.unique(scan_IDs)
    select_scan_ID = unique_scan_IDs[sample_index]

    ordered_integers = [index for index in range(len(img)) if scan_IDs[index] == select_scan_ID]
    print("ordered_integers", ordered_integers)
    num_samples = len(ordered_integers)

    if (seed != None):
        np.random.seed(seed)

    upper_bound = len(img)

    num_columns = 1
    num_rows = num_samples
    if (max_rows != None):
        num_columns = num_samples // max_rows
        num_columns = num_columns + 1 if num_samples % max_rows != 0 else num_columns
        num_rows = max_rows

    if (plot):
        fig, axes = plt.subplots(num_rows, 3*num_columns, figsize=(15*num_columns, 5 * num_rows))

    for i in range(-(-num_samples // num_rows)*num_rows):
        row = i % num_rows
        col = i // num_rows * 3

        if (i in range(num_samples)):
            index = ordered_integers[i]
            output = try_model(model, img[index], plot = False, crf = crf)
            transformed_output = output["transformed_output"]

            seg_eye = np.eye(3)[seg[index]]
            seg_eye = torch.tensor(seg_eye.transpose(2, 0, 1)).to(device)
            transformed_output_transpose = torch.tensor(transformed_output.transpose(2, 0, 1)).to(device)

            transformed_output_transpose_np = transformed_output_transpose.detach().cpu().numpy()
            seg_eye_np = seg_eye.detach().cpu().numpy()

            if (np.sum(seg_eye_np[1:]) == 0) and (np.sum(transformed_output_transpose_np[1:])/2 > 0.0001*512**2):
                true_false_positives.append(True)
            else:
                true_false_positives.append(False)

            dice_metric = 1 - dice_loss_function(transformed_output_transpose[None, ...], seg_eye[None, ...]).item()
            dice_metrics.append(dice_metric)

            if (plot):
                axes[row, col+0].imshow(transformed_output)
                axes[row, col+1].imshow(seg[index])
                axes[row, col+2].imshow(img[index])

        if (plot):
            axes[row, col+0].set_xticks([])
            axes[row, col+0].set_yticks([])
            axes[row, col+1].set_xticks([])
            axes[row, col+1].set_yticks([])
            axes[row, col+2].set_xticks([])
            axes[row, col+2].set_yticks([])

    if (plot):
        # Add titles for each column
        column_titles = ['Model', 'Ground Truth', 'Scan'] * num_columns
        for ax, title in zip(axes[0], column_titles):  # Loop over the top row of axes
            ax.set_title(title, fontsize=34)

        plt.tight_layout()

    dice_metrics = np.array(dice_metrics)
    low_count = len([x for x in dice_metrics if x < 0.7])
    false_positive_count = len([x for x in true_false_positives if x])
    print(select_scan_ID, sample_index, "Average Dice:", np.mean(dice_metrics), " | False Positives:", low_count, " | True False Positives:", false_positive_count)

    return {"DICE": np.mean(dice_metrics), "False Positives": low_count, "All DICE": dice_metrics}
    # return fig

In [ ]:
model = load_model(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'Models', '02_18_25', 'AX_T1_scheduledLR_refined_spatial_targetted_sampling_resize_DICE_wo_crf', 'models', 'most_recent_model.pth'])

In [ ]:
ordered_integers = [261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277]
for i in ordered_integers:
    print(np.max(img[271]))

In [ ]:
def try_model(model, img, seg = None, plot = True, crf = False):

    img = img.transpose((1, 0))

    if (plot):
        plt.imshow(img)
        plt.figure()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = torch.tensor(img)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.to(device)
    inputs = inputs.float()
    inputs = ScaleIntensity()(inputs)

    val_outputs = sliding_window_inference(inputs, (128, 128), 1, model) #gets the output by performing sliding_window inference on the image
    if (crf):
        val_outputs = apply_crf(val_outputs, image = inputs)
    val_outputs_numpy = val_outputs.cpu().detach().numpy()
    if (plot):
        plt.imshow(val_outputs_numpy[0][0])
        plt.figure()
        plt.imshow(val_outputs_numpy[0][1])
        plt.figure()
        plt.imshow(val_outputs_numpy[0][2])
        plt.figure()

    post_trans = Compose([Activations(softmax=True), AsDiscrete(argmax=True, to_onehot=3)])

    transform_output = post_trans(val_outputs[0])
    transform_output = transform_output.cpu().detach().numpy()
    transform_output_transposed = transform_output.transpose((2, 1, 0))
    if (plot):
        plt.imshow(transform_output_transposed)

    if (not seg is None):
        seg = seg.transpose((1, 0))
        if (plot):
            plt.figure()
            plt.imshow(seg)
        return {"transformed_output": transform_output_transposed, "output": val_outputs_numpy[0], "GT": seg}
    else:
        return {"transformed_output": transform_output_transposed, "output": val_outputs_numpy[0]}

output = try_model(model, img[270], plot = True)

In [ ]:
plt.imshow(img[271])
plt.colorbar()
plt.figure()
plt.imshow(seg[271])
plt.colorbar()

In [ ]:
fig = get_ordered_images(model, seg, img, data["id"], sample_index = 13, max_rows = 6, crf = False)

In [ ]:
AX_T1_segmenters_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'AX_T1_segmenters.txt'])
AX_T1_names = (AX_T1_segmenters_file.get().content()).decode('utf-8').split("\n")
AX_T1_names = [name for name in AX_T1_names if name != ""]
AX_T1_names = np.array(AX_T1_names)

In [ ]:
POST_AX_T1_FS_segmenters_file = navig(client, ['02_mri', '00_uchicago_data', 'Lacrimal Gland MRI for training', 'numpy_data', 'POST_AX_T1_FS_segmenters.txt'])
POST_AX_T1_FS_names = (POST_AX_T1_FS_segmenters_file.get().content()).decode('utf-8').split("\n")
POST_AX_T1_FS_names = [name for name in POST_AX_T1_FS_names if name != ""]
POST_AX_T1_FS_names = np.array(POST_AX_T1_FS_names)

In [ ]:
AX_T1_names[AX_T1_names == "Sarah "] = "Sarah"
POST_AX_T1_FS_names[POST_AX_T1_FS_names == "Sarah "] = "Sarah"
AX_T1_names[AX_T1_names == "Shrinidhi "] = "Shrinidhi"
POST_AX_T1_FS_names[POST_AX_T1_FS_names == "Shrinidhi "] = "Shrinidhi"

In [ ]:
unique_names = np.unique(np.concatenate((AX_T1_names, POST_AX_T1_FS_names)))
print(unique_names)

['Amogh' 'Mathew Illimoottil' 'Matthew H' 'Qasim' 'Sarah' 'Shrinidhi']


In [ ]:
# n = 3
DICE_coefficients_by_name = []
False_Positives_by_name = []
DICE_coefficient_per_slice_by_name = []
index = 0
for name_index in range(len(unique_names)):
    DICE_coefficients = []
    False_Positives = []
    DICE_coefficient_per_slice = []
    print(unique_names[name_index])
    select_name_indices = [i for i in range(len(AX_T1_names)) if AX_T1_names[i] == unique_names[name_index]]
    random_sample_indices = select_name_indices
    for i in random_sample_indices:
        print(index, end = " | ")
        DICE_val = get_ordered_images(model, seg, img, data["id"], sample_index = i, max_rows = 6, plot = False)
        DICE_coefficients.append(DICE_val["DICE"])
        False_Positives.append(DICE_val["False Positives"])
        DICE_coefficient_per_slice.append(DICE_val["All DICE"])
        index += 1
    False_Positives_by_name.append(False_Positives)
    DICE_coefficients_by_name.append(DICE_coefficients)
    DICE_coefficient_per_slice_by_name.append(DICE_coefficient_per_slice)

Amogh
0 | 

<ipython-input-17-1c9c81a95c49>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).to(device)


17598796 6 Average Dice: 0.8584964517124677  | False Positives: 6  | True False Positives: 5
1 | 22460223 12 Average Dice: 0.8235889655451714  | False Positives: 6  | True False Positives: 0
2 | 22543656 13 Average Dice: 0.7929728663263483  | False Positives: 5  | True False Positives: 4
3 | 27680188 18 Average Dice: 0.7901906387162148  | False Positives: 7  | True False Positives: 4
4 | 38041148 26 Average Dice: 0.8276079375508001  | False Positives: 5  | True False Positives: 5
5 | 39680487 28 Average Dice: 0.8153309771796571  | False Positives: 5  | True False Positives: 4
6 | 41084905 29 Average Dice: 0.8354193470146603  | False Positives: 6  | True False Positives: 4
7 | 49616004 36 Average Dice: 0.9101469430151801  | False Positives: 2  | True False Positives: 2
8 | 50886634 38 Average Dice: 0.8217591732637611  | False Positives: 6  | True False Positives: 4
9 | 52009570 39 Average Dice: 0.8908931544838564  | False Positives: 4  | True False Positives: 3
10 | 53892786 41 Average 

In [ ]:
# n = 3
DICE_coefficients_by_name = []
False_Positives_by_name = []
DICE_coefficient_per_slice_by_name = []
index = 0
for name_index in range(len(unique_names)):
    DICE_coefficients = []
    False_Positives = []
    DICE_coefficient_per_slice = []
    print(unique_names[name_index])
    select_name_indices = [i for i in range(len(POST_AX_T1_FS_names)) if POST_AX_T1_FS_names[i] == unique_names[name_index]]
    # random_sample_indices = random.sample(select_name_indices, n)
    random_sample_indices = select_name_indices
    for i in random_sample_indices:
        print(index, end = " | ")
        DICE_val = get_ordered_images(model, seg, img, data["id"], sample_index = i, max_rows = 6, plot = False)
        DICE_coefficients.append(DICE_val["DICE"])
        False_Positives.append(DICE_val["False Positives"])
        DICE_coefficient_per_slice.append(DICE_val["All DICE"])
        index += 1
    False_Positives_by_name.append(False_Positives)
    DICE_coefficients_by_name.append(DICE_coefficients)
    DICE_coefficient_per_slice_by_name.append(DICE_coefficient_per_slice)

Amogh
0 | 

<ipython-input-17-1c9c81a95c49>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).to(device)


17598796 Average Dice: 0.8142840536737298  | False Positives: 7
1 | 22543656 Average Dice: 0.7112643592965952  | False Positives: 10
2 | 27680188 Average Dice: 0.7685524877930932  | False Positives: 5
3 | 38041148 Average Dice: 0.8425970029150012  | False Positives: 4
4 | 41084905 Average Dice: 0.8680926908353983  | False Positives: 4
5 | 49616004 Average Dice: 0.8601518526696295  | False Positives: 4
6 | 52009570 Average Dice: 0.8839676785939746  | False Positives: 5
7 | 53892786 Average Dice: 0.9040169630229749  | False Positives: 3
8 | 54531943 Average Dice: 0.9257864554937029  | False Positives: 4
9 | 56580268 Average Dice: 0.8706209285975692  | False Positives: 5
10 | 69305171 Average Dice: 0.88806965323759  | False Positives: 3
11 | 70098571 Average Dice: 0.8958001911508588  | False Positives: 3
12 | 70663216 Average Dice: 0.8142840536737298  | False Positives: 7
13 | 78838690 Average Dice: 0.908426697027  | False Positives: 3
14 | 86566811 Average Dice: 0.8910170977595202  | Fal

In [ ]:
# Open a file to write (this will overwrite the file if it already exists)
with open('DICE_coefficients.txt', 'w') as file:
    for sublist in DICE_coefficients_by_name:
        # Convert each sublist to a string and write it to the file
        file.write(' '.join(map(str, sublist)) + '\n')

# Open a file to write (this will overwrite the file if it already exists)
with open('False_Positives_by_name.txt', 'w') as file:
    for sublist in False_Positives_by_name:
        # Convert each sublist to a string and write it to the file
        file.write(' '.join(map(str, sublist)) + '\n')

# Open a file to write (this will overwrite the file if it already exists)
with open('DICE_coefficient_per_slice_by_name.txt', 'w') as file:
    for sublist in DICE_coefficient_per_slice_by_name:
        # Convert each sublist to a string and write it to the file
        for sub_sublist in sublist:
            file.write(' '.join(map(str, sub_sublist)) + '\n')

In [ ]:
plt.boxplot(DICE_coefficients_by_name)
labels = unique_names
plt.xticks([1, 2, 3, 4, 5, 6], labels)

In [ ]:
fig

{'DICE': 0.8425970029150012}

In [ ]:
# Does one iteration of model validation
def val_iter(model, validation_dataloader, loss_fn, config = None, device = "cpu"):

    model.eval()
    outputs = {'LRT': [], 'HRT': [], 'HRI': [], "Output": [], 'max_val': [], 'LRT_shape': [], 'HRT_shape': []}
    with torch.no_grad():
        total_val_loss = 0
        val_index = 0

        for batch_index, val_data in tqdm(enumerate(validation_dataloader), desc = "val", total = len(validation_dataloader)): #load each validation dataset batch

            for sub_batch in val_data:
                val_index += 1

                HRT, HRI, LRT = sub_batch["HRT"].to(device), sub_batch["HRI"].to(device), sub_batch["LRT"].to(device) #load each image and segmentation of batch to the GPU
                output = model(LRT, HRI)

                if (batch_index == len(validation_dataloader) - 1):
                    outputs["Output"].append(output.cpu().numpy())
                    for key in sub_batch:
                        outputs[key].append(sub_batch[key].cpu().numpy())

                val_loss_value = loss_fn(output, HRT).item()
                total_val_loss += val_loss_value

        total_val_loss /= val_index

    patch_size = config["data"]["patch_size"]
    resolution_ratio = config["data"]["resolution_ratio"]

    # Reconverts model output into unnormalized numpy arrays
    for key in outputs:
        outputs[key] = np.concatenate(outputs[key], axis = 0)
    outputs = timeFirstChannel()(outputs, forward = False)
    outputs = GetAllPatches(LRT_patch_size = patch_size, resolution_ratio = resolution_ratio)(outputs, forward = False)
    outputs = Normalize()(outputs, forward = False)

    return outputs, total_val_loss